<a href="https://colab.research.google.com/github/sabnus307/My_first_repository/blob/master/cascade_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install CoolProp


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44.2 MB 1.3 MB/s 


In [ ]:
import CoolProp
print('CoolProp version:', CoolProp.__version__)
from CoolProp.CoolProp import PropsSI, get_global_param_string
import numpy as np
import matplotlib.pyplot as plt
import CoolProp.CoolProp as CP
import pandas as pd

print('High Temperature (HT) Loop Analysis')
#defining state values for HT loop
m_eg = 69.8
m_dot = 1 #assuming unit flow rate
x = 0.59 #split ratio
m_dot_HT = x  #kg/s # flowrate in HT loop
m_dot_LT = 1-x
T_E1_in_HT = 508+273.15 #K # turbine inlet temperature of Expander 1 in HT loop in Kelvin
T_EG_in = 538+273.15 #K #HT exhaust gas inlet temperature in Kelvin
T_EG_mid = 250+273.15 #K #HT exhaust gas outlet temperature in Kelvin
T_EG_out = 105+273.15 #K #HT exhaust gas outlet temperature in Kelvin
P_H = 230*1e5 #bar to Pa #cycle max pressure in Pascal
P_L = 57.3*1e5 #bar to Pa #cycle min pressure in Pascal
T_con = 20+273.15 #K #condenser temperature in kelvin
eta_pump = 0.80 #pump efficiency 
eta_turb = 0.85 #turbine efficiency
epsilon_R = 0.90 #recuperator effectiveness

#energy analysis
print('State point 1:')
P_1 = P_L
T_1 = T_con
h_1 = PropsSI( 'H','P',P_1,'T', T_1,'CO2')
s_1 = PropsSI('S','P',P_1,'T',T_1,'CO2')
print('h_1 =',h_1/1e3, 'kJ')
print('P_1 =',P_1/1e6, 'MPa')
print('T_1 =',T_1-273.15, 'C')
print('s_1 =',s_1/1e3, 'kJ/kg-K')
print('\n')

print('State point 2:')
P_2 = P_H
s_2s = s_1
h_2s = PropsSI( 'H','P',P_2,'S', s_2s,'CO2')
#eta_pump = (h_1-h_2s)/(h_1-h_2)
h_2 = h_1-((h_1-h_2s)/eta_pump)
T_2 = PropsSI( 'T','P',P_2,'H', h_2,'CO2')
s_2 = PropsSI( 'S','P',P_2,'H', h_2,'CO2')
print('h_2 =',h_2/1e3, 'kJ')
print('P_2 =',P_2/1e6, 'MPa')
print('T_2 =',T_2-273.15, 'C')
print('s_2 =',s_2/1e3, 'kJ/kg-K')
print('\n')

print('State point 4H:')
P_4H = P_H
T_4H = T_E1_in_HT
h_4H = PropsSI( 'H','P',P_4H,'T', T_4H,'CO2')
s_4H = PropsSI('S','P',P_4H,'T',T_4H,'CO2')
print('h_4H =',h_4H/1e3, 'kJ')
print('P_4H =',P_4H/1e6, 'MPa')
print('T_4H =',T_4H-273.15, 'C')
print('s_4H =',s_4H/1e3, 'kJ/kg-K')
print('\n')

print('State point 5H:')
P_5H = P_L
s_5sH = s_4H
h_5sH = PropsSI( 'H','P',P_5H,'S', s_5sH,'CO2')
#eta_turb = (h_4-h_5)/(h_4-h_5s)
h_5H = h_4H-(eta_turb*(h_4H-h_5sH))
T_5H = PropsSI( 'T','P',P_5H,'H', h_5H,'CO2')
s_5H = PropsSI('S','P',P_5H,'T',T_5H,'CO2')
print('h_5H =',h_5H/1e3, 'kJ')
print('P_5H =',P_5H/1e6, 'MPa')
print('T_5H =',T_5H-273.15, 'C')
print('s_5H =',s_5H/1e3, 'kJ/kg-K')
print('\n')

# finding Q_max
print('Rate of maximum heat exchange:')
P_6H = P_L
T_6aH = T_2 #assuming T2 = T6
h_6aH = PropsSI( 'H','P',P_6H,'T', T_6aH,'CO2')
Q_maxH = m_dot_HT*(h_5H-h_6aH) #J-kg/s #rate of maximum heat exchange
print('Q_maxHT:', Q_maxH/1e3,'kJ-kg/s')
print('\n')

print('State point 3:')
P_3H = P_H
#epsilon_R = (m_dot*(h_3-h_2))/Q_max
h_3H = h_2 + ((epsilon_R*Q_maxH)/m_dot_HT)
T_3H = PropsSI( 'T','P',P_3H,'H', h_3H,'CO2')
s_3H = PropsSI('S','P',P_3H,'H',h_3H,'CO2')
print('h_3 =',h_3H/1e3, 'kJ')
print('P_3 =',P_3H/1e6, 'MPa')
print('T_3 =',T_3H-273.15, 'C')
print('s_3 =',s_3H/1e3, 'kJ/kg-K')
print('\n')

print('State point 6:')
P_6H = P_L
#epsilon_R = (m_dotH*(h_5-h_6))/Q_maxH
h_6H = h_5H -((epsilon_R*Q_maxH)/m_dot_HT)
T_6H = PropsSI('T','P',P_6H,'H', h_6H,'CO2')
s_6H = PropsSI('S','P',P_6H,'H',h_6H,'CO2')
print('h_6 =',h_6H/1e3, 'kJ')
print('P_5 =',P_6H/1e6, 'MPa')
print('T_6 =',T_6H-273.15, 'C')
print('s_6 =',s_6H/1e3, 'kJ/kg-K')
print('\n')

print('exhaust gas calculation heater 1')
P_eg = P_L #assumption
h_eg_in = PropsSI('H','T',T_EG_in,'P',P_eg,'CO2') #enthalpy of gas at heater 1 inlet
print('h_eg_in =',h_eg_in/1e3, 'kJ')
h_eg_mid = PropsSI('H','T',T_EG_mid,'P',P_eg,'CO2') #enthalpy of gas at heater 1 outlet
print('h_eg_mid =',h_eg_mid/1e3, 'kJ')
h_diff_1 = h_eg_in - h_eg_mid #enthalpy difference in heater 1
print('h_diff_1 =',h_diff_1/1e3, 'kJ')
print('\n')

print('m_dot_CO2 Calculation')
Q_eg_1 = m_eg * h_diff_1 #heat exchange in heater 1
print('Q_eg_1 =',Q_eg_1/1e3, 'kJ')
#Heat Exchanged = m_dot * (h4- h3)
m_dot_HT_actual = Q_eg_1/ (h_4H - h_3H) # actual flowrate of CO2
print('m_dot_actual =',m_dot_actual, 'kg/s')

m_dot_LT_actual =  m_dot_LT* m_dot_HT_actual/m_dot_HT # actual flowrate of CO2 via recuperator
print('m_dot_actual =',m_dot_actual, 'kg/s')

m_dot_actual = m_dot_LT_actual + m_dot_HT_actual # actual flowrate of CO2 via heater 2
print('m_dot_actual =',m_dot_actual, 'kg/s')
print('\n')

print('exhaust gas calculation heater 2')
P_eg = P_L
print('h_eg_mid =',h_eg_mid/1e3, 'kJ')
h_eg_out = PropsSI('H','T',T_EG_out,'P',P_eg,'CO2')
print('h_eg_out =',h_eg_out/1e3, 'kJ')
h_diff_2 = h_eg_mid - h_eg_out #enthalpy difference in heater 2 for ex gas
print('h_diff_2 =',h_diff_2/1e3, 'kJ')
Q_eg_2 = m_eg * h_diff_2  #heat exchange in heater 1
print('Q_eg_2 =',Q_eg_2/1e3, 'kJ')
print('\n')

##cut
W_HTp = m_dot_HT*(h_2-h_1)  # 'MIGHT BE'common_for_both_cycle
print('Pump rate of work (input):',W_HTp/1e3,'kJ-kg/s')
W_HTt = m_dot_HT*(h_4H-h_5H)
print('High temperature Turbine rate of work (output):',W_HTt/1e3,'kJ-kg/s')
Q_HTh = m_dot_HT*(h_4H-h_3H)
print('High temperature Heater rate of work (input):',Q_HTh/1e3,'kJ-kg/s')
Q_HTc= m_dot_HT*(h_6H-h_1)
print('Condenser rate of work (output):',Q_HTc/1e3,'kJ-kg/s')
print('\n')

print('NET WORK OUTPUT (HT):',(W_HTt-W_HTp)/1e3,'kJ')

eta_cyc_HT = (W_HTt-W_HTp)/Q_HTh
print(' High temp Cycle thermal efficiency:',eta_cyc_HT)

print('\n')

###################################################################//LT LOOP//###################################################################
print('Low Temperature (LT) Loop Analysis')
#defining state values for LT loop
m_dot_LT = 1-x  #kg/s # flowrate in HT loop
T_E2_in_LT = 220.2+273.15 #K #turbine inlet temperature of Expander 2 in LT loop in Kelvin
T_EG_mid = 250+273.15 #K #LT turbine exhaust gas inlet temperature in Kelvin
T_EG_out = 105+273.15 #K #turbine exhaust gas outlet temperature in Kelvin


#energy analysis
print('State point 1:')
print('h_1 =',h_1/1e3, 'kJ')
print('P_1 =',P_1/1e6, 'MPa')
print('T_1 =',T_1-273.15, 'C')
print('s_1 =',s_1/1e3, 'kJ/kg-K')
print('\n')

print('State point 2:')
print('h_2 =',h_2/1e3, 'kJ')
print('P_2 =',P_2/1e6, 'MPa')
print('T_2 =',T_2-273.15, 'C')
print('s_2 =',s_2/1e3, 'kJ/kg-K')
print('\n')

print('State point 4L:')
P_4L = P_H
T_4L = T_E2_in_LT
h_4L = PropsSI( 'H','P',P_4L,'T', T_4L,'CO2')
s_4L = PropsSI('S','P',P_4L,'T',T_4L,'CO2')
print('h_4L =',h_4L/1e3, 'kJ')
print('P_4L =',P_4L/1e6, 'MPa')
print('T_4L =',T_4L-273.15, 'C')
print('s_4L =',s_4L/1e3, 'kJ/kg-K')
print('\n')

print('State point 5L:')
P_5L = P_L
s_5sL = s_4L
h_5sL = PropsSI( 'H','P',P_5L,'S', s_5sL,'CO2')
#eta_turb = (h_4-h_5)/(h_4-h_5s)
h_5L = h_4L-(eta_turb*(h_4L-h_5sL))
T_5L = PropsSI( 'T','P',P_5L,'H', h_5L,'CO2')
s_5L = PropsSI('S','P',P_5L,'T',T_5L,'CO2')
print('h_5L =',h_5L/1e3, 'kJ')
print('P_5L =',P_5L/1e6, 'MPa')
print('T_5L =',T_5L-273.15, 'C')
print('s_5L =',s_5L/1e3, 'kJ/kg-K')
print('\n')

print('Rate of maximum heat exchange:')
P_6L = P_L
T_6La = T_2 #assuming T2 = T6
h_6La = PropsSI( 'H','P',P_6L,'T', T_6La,'CO2')
Q_max_L = m_dot_LT*(h_5L-h_6La) #J-kg/s #rate of maximum heat exchange
print('Q_maxLT:', Q_max_L/1e3,'kJ-kg/s')
print('\n')

print('State point 3L:')
P_3L = P_H
#epsilon_R = (m_dot*(h_3-h_2))/Q_max
h_3L = h_2 + ((epsilon_R*Q_max_L)/m_dot_LT)
T_3L = PropsSI( 'T','P',P_3L,'H', h_3L,'CO2')
s_3L= PropsSI('S','P',P_3L,'H',h_3L,'CO2')
print('h_3L =',h_3L/1e3, 'kJ')
print('P_3L =',P_3L/1e6, 'MPa')
print('T_3L =',T_3L-273.15, 'C')
print('s_3L =',s_3L/1e3, 'kJ/kg-K')
print('\n')

print('State point 6L:')
P_6L = P_L
#epsilon_R = (m_dot*(h_5-h_6))/Q_max
h_6L = h_5L -((epsilon_R*Q_max_L)/m_dot_LT)
T_6L = PropsSI('T','P',P_6L,'H', h_6L,'CO2')
s_6L = PropsSI('S','P',P_6L,'H',h_6L,'CO2')
print('h_6L =',h_6L/1e3, 'kJ')
print('P_5L =',P_6L/1e6, 'MPa')
print('T_6L=',T_6L-273.15, 'C')
print('s_6L =',s_6L/1e3, 'kJ/kg-K')
print('\n')

W_HTp = m_dot_HT_actual*(h_2-h_1)  # 'MIGHT BE'common_for_both_cycle
print('Pump rate of work (input):',W_HTp/1e3,'kJ-kg/s')
W_HTt = m_dot_HT_actual*(h_4H-h_5H)
print('High temperature Turbine rate of work (output):',W_HTt/1e3,'kJ-kg/s')
Q_HTh = m_dot_HT_actual*(h_4H-h_3H)
print('High temperature Heater rate of work (input):',Q_HTh/1e3,'kJ-kg/s')
Q_HTc= m_dot_HT_actual*(h_6H-h_1)
print('Condenser rate of work (output):',Q_HTc/1e3,'kJ-kg/s')
print('\n')

print('NET WORK OUTPUT (HT):',(W_HTt-W_HTp)/1e3,'kJ')

eta_cyc_HT = (W_HTt-W_HTp)/Q_HTh
print(' High temp Cycle thermal efficiency:',eta_cyc_HT)

print('\n')

W_LTp = m_dot_LT_actual*(h_2-h_1) 
print('Pump rate of work (input):',W_LTp/1e3,'kJ-kg/s')
W_LTt = m_dot_LT_actual*(h_4L-h_5L)
print('LOW temperature Turbine rate of work (output):',W_LTt/1e3,'kJ-kg/s')
Q_LTh = m_dot_LT_actual*(h_4L-h_3L)
print('LOW temperature Heater rate of work (input):',Q_LTh/1e3,'kJ-kg/s')
Q_LTc= m_dot_LT_actual*(h_6L-h_1)
print('condenser rate of work (output):',Q_LTc/1e3,'kJ-kg/s')
print('\n')

print('NET WORK OUTPUT (LT):',(W_LTt-W_LTp)/1e3,'kJ')

eta_cyc_LT = (W_LTt-W_LTp)/Q_LTh
print(' LOW temp Cycle thermal efficiency:',eta_cyc_LT)

print('\n')


############################################################/// TOTAL ///#######################################################################
#overall efficiency calculation
print('///////////////////////TOTAL CYCLE CONSIDERATION//////////////////')
print('\n')
W_p = W_LTp + W_HTp
print('Pump rate of work (input):',W_p/1e3,'kJ-kg/s')
W_t = W_HTt + W_LTt
print(' Turbine rate of work (output):',W_t/1e3,'kJ-kg/s')
W_net = W_t - W_p
print(' Net rate of work (output):',W_net/1e3,'kJ-kg/s')
Q_h = Q_HTh + Q_LTh
print(' Heater rate of work (input):',Q_h/1e3,'kJ-kg/s')
Q_c = Q_HTc + Q_LTc
print('condenser rate of work (output):',Q_c/1e3,'kJ-kg/s')

print('NET WORK OUTPUT (TOTAL cycle):',(W_t-W_p)/1e3,'kJ')

eta_cyc = (W_t-W_p)/Q_h
print('Cycle thermal efficiency:',eta_cyc)

Q_eg_max = 40.9e6 #W
m_eg = 69.8 #kg/s #exhaust gas mass flow rate
h_diff = Q_eg_max/m_eg #enthalpy difference

#Q_EG = (h_4H-h_3H)+(h_4L-h_3L)

#Q_EG1 = 22719e3 #TABLE
#Q_EG2 = 11467e3 #TABLE
#Q_EG = Q_EG1 + Q_EG2
#Q = m*Cp*dT #Cp= 2.3 #kJ/kg-K #for natural gas

print('\n')

P_eg = P_L
T_eg_in = T_EG_in
h_eg_in = PropsSI( 'H','P',P_eg,'T', T_eg_in,'CO2')
h_eg_out = PropsSI( 'H','P',P_eg,'T', T_EG_out,'CO2')
print('h_eg_in:', h_eg_in/1e3,'kJ-kg/s')
print('h_eg_out:', h_eg_out/1e3,'kJ-kg/s')
eta_hr = (h_eg_in- h_eg_out)/h_diff
print('Heat recovery efficiency:',eta_hr)

print('\n')

#print('WASTE GAS WORK:',Q_EG*69.8/1e3)
#eta_hr = Q_EG/h_diff
#print('Heat recovery efficiency:',eta_hr)
n_sys = eta_hr*eta_cyc
print('System thermal efficiency:',n_sys)
print('\n')


#values from paper
eta_cyc_ref = 0.282
eta_hr_ref = 0.836 
n_sys_ref = 0.236
W_net_ref = 9647e3


#error calculation
eta_cyc_error = abs(eta_cyc -eta_cyc_ref)
print('Cycle thermal efficiency error:',round(eta_cyc_error *100/eta_cyc_ref,2),'%')

eta_hr_error = abs(eta_hr -eta_hr_ref)
print('Heat Recovery efficiency error',round(eta_hr_error *100/eta_hr_ref,2),'%')

W_net_error = abs(W_net - W_net_ref)
print('Net work error:',round(W_net_error *100/W_net_ref,2),'%')

n_sys_error = abs(n_sys - n_sys_ref)
print('Sytem efficiency error:',round(n_sys_error *100/n_sys_ref,2),'%')

CoolProp version: 6.4.1
High Temperature (HT) Loop Analysis
State point 1:
h_1 = 255.86249820761472 kJ
P_1 = 5.73 MPa
T_1 = 20.0 C
s_1 = 1.1877062900124837 kJ/kg-K


State point 2:
h_2 = 282.12794013842154 kJ
P_2 = 23.0 MPa
T_2 = 43.749426219845134 C
s_2 = 1.204347085292089 kJ/kg-K


State point 4H:
h_4H = 980.7459683323385 kJ
P_4H = 23.0 MPa
T_4H = 508.0 C
s_4H = 2.645919009557859 kJ/kg-K


State point 5H:
h_5H = 825.4246669096102 kJ
P_5H = 5.73 MPa
T_5H = 360.1052405418012 C
s_5H = 2.690053666735606 kJ/kg-K


Rate of maximum heat exchange:
Q_maxHT: 214.16862889748654 kJ-kg/s


State point 3:
h_3 = 608.8258486261129 kJ
P_3 = 23.0 MPa
T_3 = 214.29748610644208 C
s_3 = 2.0464900833073565 kJ/kg-K


State point 6:
h_6 = 498.72675842191876 kJ
P_5 = 5.73 MPa
T_6 = 69.19368678683628 C
s_6 = 1.9965885465947304 kJ/kg-K


exhaust gas calculation heater 1
h_eg_in = 1032.3943401305903 kJ
h_eg_mid = 702.6889662334919 kJ
h_diff_1 = 329.70537389709847 kJ


m_dot_CO2 Calculation
Q_eg_1 = 23013.4350980